Exploring Material Parameterization

In [1]:
from qd.cae.dyna import D3plot
from importlib import reload

import pandas as pd
import numpy as np

import Utils
import EdaTools

# get jupyter to use more of my screen space
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# improve inline plots
%matplotlib widget

# import the representative D3plot data

In [3]:
path = "/media/martin/Stuff/research/MaterailModels/MM008/"
file = "MM008_job01.d3plot"

d3plot_HD = D3plot(path + file, read_states="disp")

path = "/media/martin/Stuff/research/MaterailModels/MM007/"
file = "MM007_job01.d3plot"

d3plot_LD = D3plot(path + file, read_states="disp")

# Extract surface nodes

In [4]:
reload(Utils)

surface_node_extractor = Utils.SurfaceNodeIds(threshold=17)
node_ids_surface = surface_node_extractor.fit_transform(d3plot_HD)
print(node_ids_surface)

node_list = list(node_ids_surface)

<generator object SurfaceNodeIds.transform at 0x7f8540ac4fc0>


## plot the initial geometry

In [5]:
reload(Utils)
reload(EdaTools)
%matplotlib widget

EdaTools.plot_3d(
    Utils.nodes_to_coord_array(
        d3plot_HD.get_nodeByID(
            node_list)))

/home/martin/anaconda3/envs/musical-dollop/lib/python3.6/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecoginized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
/home/martin/anaconda3/envs/musical-dollop/lib/python3.6/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecoginized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## plot the final geometry

In [ ]:
reload(Utils)
reload(EdaTools)

EdaTools.plot_3d(
    Utils.nodes_to_coord_array(
        d3plot_HD.get_nodeByID(node_list), timestep=-1))

## plot displacement vectors

In [ ]:
reload(Utils)
reload(EdaTools)

EdaTools.plot_disp_quiver(d3plot_HD, node_list=node_list)

# Analyze Nodal properties

In [ ]:
reload(Utils)
reload(EdaTools)

EdaTools.plot_disp_hist(d3plot_HD, time_step=-1, num_bins=30)

# Analyze element properties

## Element Stress distribution

In [ ]:
reload(Utils)
reload(EdaTools)

EdaTools.plot_stress_hist(d3plot_HD, time_step=-1, num_bins=50)

## Element strain distribution

In [ ]:
reload(Utils)
reload(EdaTools)

EdaTools.plot_strain_hist(d3plot_HD, time_step=-1, num_bins=50)

# Extract 'experimental DIC' results from the HD FEM
Extract a portion of the surface nodes to be tracked as if they are DIC data.

In [ ]:
reload(Utils)
reload(EdaTools)

node_ids_dic = Utils.extract_observed_surface(d3plot_HD, time_step=0, eps=0.001)

In [ ]:
EdaTools.plot_3d(
    Utils.nodes_to_coord_array(
        d3plot_HD.get_nodeByID(node_ids_dic), timestep=0))

# Fit a model to the 'DIC' data from the HD FEM
* predict u, v, w given x, y, z and time index
* construct a longform dataframe containing observations for each timestep
* 

In [ ]:
# create pandas frame

num_time_steps = result_d3plot.get_nTimesteps()

data = []

# for each node on the observed surface
for nid in node_ids_dic:
    # Extract the node results
    node = result_d3plot.get_nodeByID(nid)
    # extract the initial coordinates
    cords = node.get_coords()[0]
    # for each timestep
    for ts, vec in enumerate(node.get_disp()):
        # estract the node displacement and create and observation vector
        data.append(np.append(cords, [ts, *vec]))

data_df = pd.DataFrame(data, columns=["x", "y", "z", 't_index', "u", "v", "w"])

In [ ]:
data_df.head()

In [ ]:
data_df.describe()

In [ ]:
from sklearn import svm
X = data_df[["x", "y", "z", 't_index']]
y = data_df["u"]
clf = svm.SVR()
clf.fit(X, y)

In [ ]:
from sklearn.metrics import r2_score

y_pred = clf.predict(X)
r2_score(y, y_pred)

In [ ]:
clf.score(X, y)

# TODO


* Fit a model to this data 
* create a second model with lower mesh density to represent the simulation 
* Map the 'DIC' data to the 'FEM' for a best fit.


In [ ]:
from scipy.interpolate import Rbf

rbf = Rbf(data_df.x, data_df.y, data_df.z, epsilon=2)
